In [2]:
import pandas as pd
from pymongo import MongoClient
import re
from haversine import haversine

In [ ]:
#!pip3 install --user haversine

In [3]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [4]:
db, coll = connectCollection('companies','airports')
db, coll1 = connectCollection('companies','preferred')

In [5]:
airports = list(coll.find({"type": 'large_airport'}))

In [6]:
business = list(coll1.find({"$and":[{"money_raised":{"$ne":"$0"}},{"year":{"$gte":2009}}]}))

In [ ]:
#business = list(coll1.find({"distance":{"$exists":True}}))

In [ ]:
'''
,{"money_raised":{"$regex":"k$"}}
import re
for noMatch in db.inventory.find( { "item": { "$not": re.compile("^p.*") } } ):
    print noMatch
'''

In [ ]:
print(len(airports))

In [ ]:
print(len(business))

In [ ]:
business[0]['location']['coordinates'][0]

In [ ]:
name=[]
for bus in business:
    name.append(bus['name'])
    

In [ ]:
business[0]['location']['coordinates'][1]

In [ ]:
airports[0]

In [ ]:
#long 0
airports[0]['location']['coordinates'][0]

In [ ]:
#lat 1
airports[0]['location']['coordinates'][1]

In [ ]:
def getcoord(query,lat,long):
    for que in query: 
        lat.append(que['location']['coordinates'][1])
        long.append(que['location']['coordinates'][0])
    return len(lat),len(long)       

In [ ]:
lat1=[]
long1=[]
getcoord(business,lat1,long1)

In [ ]:
bus=list(zip(lat1,long1))

In [ ]:
lat2=[]
long2=[]
getcoord(airports,lat2,long2)

In [ ]:
air=list(zip(lat2,long2))

In [ ]:
# (lat, lon)
#haversine((45.7597, 4.8422),(48.8567, 2.3508))
#243.71209416020253

In [ ]:
l=[1,2,3,5]
min(l)

In [ ]:
def harver(elem,zipi):
    comparator=[]
    for a in air:
        comparator.append(haversine(elem,a))
    return min(comparator)

In [ ]:
futcol=[]
#futcol tiene que tener 99 elemento
for i in bus:
    futcol.append(harver(i,air))

In [ ]:
dfair=pd.DataFrame({"name":name,"long":long1,"lat":lat1,"dis to an airport":futcol})

In [ ]:
actuali=list(zip(business,futcol))

In [ ]:
for i in actuali[:]:
    print(i[0])

In [ ]:
for i in actuali:
    value={"$set":{'distance':i[1]}}
    coll1.update_one(i[0],value)

In [ ]:
cutoffs = [0,10,20,50,100,300]
km_labels = ['0-10km', '10-20km', '20-50km','50-100km','100-300km']
punt['range'] = pd.cut(punt['dis to an airport'], cutoffs, labels = km_labels)
punt['range'].value_counts()

In [ ]:
dfair

In [ ]:
export_csv = dfair.to_csv('distoair.csv', index=None, header=True)

In [ ]:
#getAirLoc(airports[0])

In [ ]:
#for air in airports:
    #value={"$set":{'location':getAirLoc(air)}}
    #coll.update_one(air,value)   

# GEOQUERIES

In [166]:
db, coll = connectCollection('companies','starbucks')
db, coll1 = connectCollection('companies','preferred')

In [167]:
star= list(coll.find())

In [73]:
business = list(coll1.find({"$and":[{"money_raised":{"$ne":"$0"}},{"year":{"$gte":2009}}]}))

In [168]:
star[0]

{'_id': ObjectId('5dd672703fe4b24ba1eaecf0'),
 'name': 'Starbucks',
 'long': -73.9858802,
 'lat': 40.7567274,
 'location': {'type': 'Point', 'coordinates': [-73.9858802, 40.7567274]}}

In [7]:
business[0]

{'_id': ObjectId('5dd05cc6754df22b96b8c84b'),
 'Id': '52cdef7c4bab8bd675297f94',
 'name': 'PeekYou',
 'category': 'search',
 'description': '',
 'long': -73.985506,
 'year': 2012,
 'money_raised': '$1.83M',
 'location': {'type': 'Point', 'coordinates': [-73.985506, 40.757929]},
 'distance': 9.747111732704937}

In [158]:
def geoquery(geoindex,distance,col):
    points= coll.find(
        {"location": 
         {"$near": 
          {"$geometry":
           geoindex,
           "$maxDistance":distance
            }
        }
    })
    return list(points)
#que la función me devuelva la len(list(points))

In [182]:
#coll de whatever
#coll1 de empresas
comparatorDosPuntoCero=[]
for bu in business:
    comparatorDosPuntoCero.append(geoquery(bu['location'],100,coll))


In [183]:
cuantos=[]
for i in comparatorDosPuntoCero:
    cuantos.append(len(i))
print(len(cuantos))

99


In [60]:
#traer la columna NO SORTED
data = pd.read_csv("master99.csv") 
data.head()

name        long        lat  dis to an airport    range  \
0     PeekYou  -73.985506  40.757929           9.747112   0-10km   
1       GENWI -111.903500  33.817100          43.734956  20-50km   
2       Fixya -122.323895  37.566879           7.339122   0-10km   
3   Widgetbox -122.398599  37.798853          17.835823  10-20km   
4  RazorGator -118.445243  34.047312          12.149767  10-20km   

   airports 40km  airports 10km  starbucks 500m  starbucks 1km  drunk af 500m  \
0              3              1              16             56             65   
1              0              0               0              0              0   
2              2              1               4              4             87   
3              2              0               6             16              7   
4              1              0               3              4              0   

   drunk af 1km  schools 5km  veggies 100m  veggies 500m  
0           130          376             0             6  
1             0            6             0             0  
2            87          128            16            76  
3            27          148             0             1  
4            10           98             0             1

In [184]:
data["starbucks 100m"]=cuantos

In [186]:
data=data.drop(["party 100m"], axis=1)

In [187]:
data.head()
#raro pero si, hay 22 veggies a 500m de Fixya<<

name        long        lat  dis to an airport    range  \
0     PeekYou  -73.985506  40.757929           9.747112   0-10km   
1       GENWI -111.903500  33.817100          43.734956  20-50km   
2       Fixya -122.323895  37.566879           7.339122   0-10km   
3   Widgetbox -122.398599  37.798853          17.835823  10-20km   
4  RazorGator -118.445243  34.047312          12.149767  10-20km   

   airports 40km  airports 10km  schools 5km  schools 2km  veggies 500m  \
0              3              1           88           17             3   
1              0              0            5            2             0   
2              2              1           40           21            22   
3              2              0           54           12             0   
4              1              0           45           16             3   

   veggies 100m  party 1km  party 500m  starbucks 500m  starbucks 100m  
0             0         26           9               9               1  
1             0          0           0               0               0  
2             6         26          25               1               0  
3             0         12           3               4               0  
4             0         11           1               1               1

In [188]:
export_csv = data.to_csv('master99.csv', index=None, header=True)

In [61]:
data=data.drop(['starbucks 500m', 'starbucks 1km', 'drunk af 500m', "drunk af 1km", "schools 5km", 'veggies 100m', 'veggies 500m'], axis=1)

In [ ]:
data

In [ ]:
{"location":{"$near":{"$geometry":{'type': 'Point', 'coordinates': [-122.323895, 37.566879]},"$maxDistance":100}}}
         